In [23]:
import pymongo
from pymongo import MongoClient
import datetime
import json

In [18]:
client = MongoClient('172.20.26.172', 27017)
db=client['rawData']
db.collection_names()

[]

In [20]:
collection = db.wowprimeipeen

In [34]:
with open("./data/diendata.json","r") as d:
    diendata=json.load(d)

In [25]:
collection.insert_many(diendata)

In [32]:
x=list(collection.find({}))

[{'Ncomment': 2,
  '_id': ObjectId('5a15363e80b11022c4b3a3d7'),
  'address': '台南市中西區中山路101號',
  'averagecost': None,
  'bigstyle': '咖啡、簡餐、茶',
  'collecount': 16,
  'id': 1040602,
  'ipeenscore': 45,
  'lat': 22.99447701949,
  'lng': 120.20760100484,
  'name': '0.618',
  'reviewdate': '2016/04/05',
  'smallstyle': '複合式',
  'status': '正常營業',
  'tele': '062210217',
  'viewcount': 15758},
 {'Ncomment': 4,
  '_id': ObjectId('5a15363e80b11022c4b3a3d8'),
  'address': '雲林縣斗六市仁義路106號',
  'averagecost': 225,
  'bigstyle': '咖啡、簡餐、茶',
  'collecount': 3,
  'id': 1023458,
  'ipeenscore': 45,
  'lat': 23.673847314249997,
  'lng': 120.53735501095001,
  'name': '018咖啡甜品玩國',
  'reviewdate': '2015/06/22',
  'smallstyle': '複合式',
  'status': '正常營業',
  'tele': None,
  'viewcount': 6742},
 {'Ncomment': 5,
  '_id': ObjectId('5a15363e80b11022c4b3a3d9'),
  'address': '新竹縣竹北市自強七街48號1樓',
  'averagecost': 289,
  'bigstyle': '異國料理',
  'collecount': 27,
  'id': 1016944,
  'ipeenscore': 45,
  'lat': 24.81910820094999

In [33]:
collection.drop()